# Evaluate Similarity Grouping


In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from models import ModelMgr
from models.embedding.SentenceTransformer import SentenceTransformerEmbeddingModel
from models.semantic_validation import LLaMAValidationModel

from db.operators import Dummy, Select
from db.criteria import SoftEqual
from db.structure import Column, Constant
import kagglehub

from util import calculate_metrics

import time

[nltk_data] Downloading package punkt to /home/nico/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/nico/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
m = ModelMgr()
stem = SentenceTransformerEmbeddingModel(m)
lsv = LLaMAValidationModel(m)
# lsv = DeepSeekValidationModel(m)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
#system_prompt = "You are an expert in checking semantic equivalence. Respond with \"yes\" and \"no\" only!"
#prompt_template = "Is \"{a}\" semantically equal to \"{b}\"?"

system_prompt = "Respond with \"yes\"/ \"no\" only!"
prompt_template = 'Can "{}" be referred as  "{}"'

In [4]:
path = kagglehub.dataset_download("lakritidis/product-classification-and-categorization")

df = pd.read_csv(f"{path}/pricerunner_aggregate.csv", header=None, index_col=0)
df.drop(columns=[2, 3, 5], inplace=True)
df.columns = ["product", "category_1", "category_2"]
df.head()

,product,category_1,category_2
0,,,
1,apple iphone 8 plus 64gb silver,Apple iPhone 8 Plus 64GB,Mobile Phones
2,apple iphone 8 plus 64 gb spacegrau,Apple iPhone 8 Plus 64GB,Mobile Phones
3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,Apple iPhone 8 Plus 64GB,Mobile Phones
4,apple iphone 8 plus 64gb space grey,Apple iPhone 8 Plus 64GB,Mobile Phones
5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,Apple iPhone 8 Plus 64GB,Mobile Phones


In [5]:
categories = list(df["category_2"].unique())
print(categories)

['Mobile Phones', 'TVs', 'CPUs', 'Digital Cameras', 'Microwaves', 'Dishwashers', 'Washing Machines', 'Freezers', 'Fridge Freezers', 'Fridges']


In [6]:
df_reduced = pd.concat([df[df["category_2"] == category].sample(10) for category in categories])
gt = {(x["product"], x["category_2"]) for key, x in df_reduced.iterrows()}
print(str(gt)[0:100], "...", len(gt))

{('russell hobbs rhm2064g compact solo microwave grey', 'Microwaves'), ('indesit dsr15bk slimline di ... 100


In [7]:
overall_result = {}

def evaluate(method, threshold, fetch_one = False, calc_bleu = False):
    pred = []
    runtimes = []
    for category in tqdm(categories):
        d = Dummy("products", ["product"], [(x[0], ) for x in gt])
        s = Select(d, SoftEqual(Column("product"), Constant(category), method=method, em=stem, sv=lsv, threshold=threshold, zfs_system_prompt=system_prompt, zfs_prompt_template = prompt_template))

        tic = time.time()
        if fetch_one:
            result = [s.open().fetch_one()]
        else:
            result = s.open().fetch_all()
        toc = time.time()

        pred.extend([(x["product"], category) for x in result])
        runtimes.append(toc - tic)

    pred = set(pred)

    scores = calculate_metrics(gt, pred, calc_bleu=calc_bleu)
    scores["Runtime"] = np.mean(runtimes)

    print(scores)

    return scores, pred

In [8]:
for thresh in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("threshold", thresh, False, True)
    overall_result[("threshold", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 10/10 [00:04<00:00,  2.40it/s]


{'Precision': 0.10675381263616558, 'Recall': 0.98, 'F1 Score': 0.1925343811394892, 'bleu1': 0.9980909090909091, 'bleu2': 0.9979896122473321, 'bleu3': 0.9978719262778781, 'bleu4': 0.9977326571860027, 'Runtime': 0.4159417629241943}


100%|██████████| 10/10 [00:03<00:00,  3.02it/s]


{'Precision': 0.10738255033557047, 'Recall': 0.96, 'F1 Score': 0.193158953722334, 'bleu1': 0.9958505971793201, 'bleu2': 0.9954864951319502, 'bleu3': 0.9951630778719928, 'bleu4': 0.9947552641197736, 'Runtime': 0.33110885620117186}


100%|██████████| 10/10 [00:03<00:00,  2.98it/s]


...

{'Precision': 0.6451612903225806, 'Recall': 0.2, 'F1 Score': 0.30534351145038163, 'bleu1': 0.48125943559400136, 'bleu2': 0.4099488982393146, 'bleu3': 0.3616004982584514, 'bleu4': 0.33162421874276204, 'Runtime': 0.3836843252182007}


100%|██████████| 10/10 [00:03<00:00,  2.64it/s]


{'Precision': 1.0, 'Recall': 0.01, 'F1 Score': 0.019801980198019802, 'bleu1': 0.14078107292873082, 'bleu2': 0.06784549570954383, 'bleu3': 0.05054231645060737, 'bleu4': 0.037295369118533546, 'Runtime': 0.3785877227783203}


100%|██████████| 10/10 [00:03<00:00,  2.90it/s]

{'Precision': 0, 'Recall': 0.0, 'F1 Score': 0, 'bleu1': -1, 'bleu2': -1, 'bleu3': -1, 'bleu4': -1, 'Runtime': 0.3439875841140747}


In [9]:
res, pred = evaluate("zero-few-shot", None, False, True)
overall_result[("zero-few-shot", None)] = res

100%|██████████| 10/10 [00:26<00:00,  2.63s/it]


{'Precision': 0.7241379310344828, 'Recall': 0.42, 'F1 Score': 0.5316455696202531, 'bleu1': 0.6686946667770143, 'bleu2': 0.6088971052118335, 'bleu3': 0.5560174999684565, 'bleu4': 0.5190346640167757, 'Runtime': 2.6299542665481566}


In [10]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("both", thresh, False, True)
    overall_result[("both", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 10/10 [11:16<00:00, 67.68s/it]


{'Precision': 0.8333333333333334, 'Recall': 0.45, 'F1 Score': 0.5844155844155844, 'bleu1': 0.6794564658436427, 'bleu2': 0.6201113360556711, 'bleu3': 0.569423208084529, 'bleu4': 0.5369079675514906, 'Runtime': 67.67934219837188}


100%|██████████| 10/10 [03:38<00:00, 21.86s/it]


{'Precision': 0.803921568627451, 'Recall': 0.41, 'F1 Score': 0.543046357615894, 'bleu1': 0.6541716633060619, 'bleu2': 0.6027290750611993, 'bleu3': 0.5553724229062721, 'bleu4': 0.5206783911403058, 'Runtime': 21.86234359741211}


100%|██████████| 10/10 [02:10<00:00, 13.01s/it]


...

{'Precision': 0.9310344827586207, 'Recall': 0.27, 'F1 Score': 0.4186046511627907, 'bleu1': 0.4960495878448114, 'bleu2': 0.4264709319702338, 'bleu3': 0.3716630272788912, 'bleu4': 0.33507127083178195, 'Runtime': 8.670647740364075}


100%|██████████| 10/10 [00:36<00:00,  3.70s/it]


{'Precision': 0.7222222222222222, 'Recall': 0.13, 'F1 Score': 0.22033898305084748, 'bleu1': 0.4184024319273668, 'bleu2': 0.334317327567578, 'bleu3': 0.28064236169840034, 'bleu4': 0.24135212969021438, 'Runtime': 3.6956775903701784}


100%|██████████| 10/10 [00:04<00:00,  2.30it/s]

{'Precision': 0, 'Recall': 0.0, 'F1 Score': 0, 'bleu1': -1, 'bleu2': -1, 'bleu3': -1, 'bleu4': -1, 'Runtime': 0.4351715803146362}


In [21]:
df_results = pd.DataFrame([{"method": k[0], "threshold": k[1]} | v for k,v in overall_result.items()])
df_results.head()

,method,threshold,Precision,Recall,F1 Score,bleu1,bleu2,bleu3,bleu4,Runtime
0,threshold,0.01,0.106754,0.98,0.192534,0.998091,0.997990,0.997872,0.997733,0.415942
1,threshold,0.02,0.107383,0.96,0.193159,0.995851,0.995486,0.995163,0.994755,0.331109
2,threshold,0.03,0.109840,0.96,0.197125,0.995851,0.995486,0.995163,0.994755,0.335672
3,threshold,0.04,0.110190,0.93,0.197034,0.992166,0.991644,0.991004,0.990187,0.336389
4,threshold,0.05,0.114815,0.93,0.204396,0.992166,0.991644,0.991004,0.990187,0.348533


In [22]:
df_results.to_csv("results/EvaluateSemanticFiltering.csv")